# 问答中的tokenizer
从文本中找到回答问题的片段，需要找到片段的开始和结束的位置

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

当文本长度过长导致无法完全输入需要进行截断，此时只截断文本而不去截断问题

In [ ]:
inputs = tokenizer(question, 
                   long_context, 
                   max_length=384,
                   truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

上述截断文本的方法容易导致截断的文本不完整，因此需要对截断的文本进行处理，使得截断的文本能够完整的表达原文本的意思，下述方法将文本分为小块，小块之间有重叠部分，这样截断的文本就能够完整的表达原文本的意思

In [ ]:
sentence = "This sentence is not too long but we are going to split it anyway."
inputs = tokenizer(sentence, 
                   truncation=True, 
                   return_overflowing_tokens=True, 
                   max_length=6, 
                   stride=2)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

可以添加padding使得每个小块的长度相同，这样可以方便进行batch训练

In [ ]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)